In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gpt_wrapper
from gpt_wrapper.chat import Chat

gpt_wrapper.api_base = "http://mnlp-backend-938795011.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "3e510581-28fa-4342-9758-4fa131bc2f42"

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

# Reward shaping function
def reward(gpt4_evaluation, action):
    if action == "FETCH":
        return 0.1
    else: 
        if gpt4_evaluation == 'Good':
            return 2  # Positive reward for good evaluation without fetching
        elif gpt4_evaluation == 'Bad':
            return -1  # Negative reward for bad evaluation

# Simulate the environment
def environment_step(question, answer, chat):
    prompt = "Here is the question:" + question + ". And here is the given answer:" + answer + ". Do you think it was a good or bad answer?"
    evaluation = str(chat.ask(prompt = prompt, instruction = "Answer with one word, either 'good' or 'bad'. Do not give an explanation."))
    return evaluation

# Training the policy network
def train_policy_network(queries, policy_network, optimizer):
    gamma = 0.1  # discount factor for future rewards
    for epoch in range(100):
        chat = Chat.create("chat")
        state = queries[0]
        rewards = []
        log_probs = []
        for query in queries:
            # Forward pass
            probs = policy_network(state)
            m = torch.distributions.Categorical(probs)
            action_index = m.sample()
            action = 'FETCH' if action_index.item() == 0 else 'NO_FETCH'
            results = db.similarity_search_with_score(query_text, k = 5)
            context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
            
            if action == 'FETCH':
                state += context_text
            
            answer = str(chat.ask(content = state))
            
            # Simulate the environment step
            gpt4_evaluation = environment_step(query, answer, chat)
            reward = reward(gpt4_evaluation, action)
            rewards.append(reward)
            
            # Calculate the log probability
            log_prob = m.log_prob(action_index)
            log_probs.append(log_prob)
            
            # Update the state based on the action taken
            state = torch.tensor([1, 0] if action == 'FETCH' else [0, 1], dtype=torch.float)

        # Update policy
        discounted_rewards = []
        cumulative_rewards = 0
        for reward in rewards[::-1]:
            cumulative_rewards = reward + gamma * cumulative_rewards
            discounted_rewards.insert(0, cumulative_rewards)

        discounted_rewards = torch.tensor(discounted_rewards)
        policy_loss = []
        for log_prob, discounted_reward in zip(log_probs, discounted_rewards):
            policy_loss.append(-log_prob * discounted_reward)

        optimizer.zero_grad()
        policy_loss = torch.stack(policy_loss).sum()
        policy_loss.backward()
        optimizer.step()

# Instantiate the network and optimizer
input_size = 2  # Example: size of the state vector
hidden_size = 128
output_size = 2  # Two actions: FETCH or NO_FETCH
policy_network = PolicyNetwork(input_size, hidden_size, output_size)
optimizer = optim.Adam(policy_network.parameters(), lr=0.01)

# Example queries sequence
queries = ['initial query', 'follow-up', 'new topic', 'follow-up']
train_policy_network(queries, policy_network, optimizer)

TypeError: environment_step() missing 1 required positional argument: 'chat'

In [10]:
chat = Chat.create("chat")
prompt = "Here is the question:" + "What is a heat pump" + ". And here is the given answer:" + "A heat pump is a type of animal" + ". Do you think it was a good or bad answer?"
evaluation = str(chat.ask(content = prompt, instruction = "Answer with one word, either 'good' or 'bad'. Do not give an explanation."))
print(reward(evaluation, False))


-1
